In [ ]:
pip install tensorflow

In [8]:
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

- **Problem** : I have an imbalanced dataset with 500 original videos and 5,000 fake videos that I want to use for deepfake detection. Extracting frames from each video would result in far more fake frames, skewing the training distribution.
- **Solution** : To mitigate this class imbalance while retaining diversity, I can restrict the number of frames per fake video. For example, sampling only 10 frames evenly across each fake video results in 5,000 fake frames to match the 500 original videos. This maintains equal class representation during training.
- Another possible solution is data augmentation but considering the storage in mind this is the best possible option we have if this doesn't work then we will use data augmentation.

- This the V2 of the code for deep fake detector
- main change introduced we introduced the capability to split the videos into frames and selecting the frames based on variance rather than selecting them randomly
- The key is to ensure that these frames are representative of the entire video. You can select frames evenly spaced throughout the video, or use a more sophisticated method like selecting frames that have the most variance or are most representative of the content.

## video ---> frame splitting

In [3]:
import cv2
import os
import numpy as np

def extract_faces_with_variance(video_path, output_folder, label, max_frames=None):
    video_name = os.path.splitext(os.path.basename(video_path))[0]

    try:
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Unable to open video: {video_path}")
            return

        frame_variance = []

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            var = np.var(gray_frame)
            frame_variance.append((var, frame))

        cap.release()

        # Select frames based on the max_frames parameter
        if max_frames and max_frames > 0:
            frame_variance.sort(key=lambda x: x[0], reverse=True)
            selected_frames = frame_variance[:max_frames]
        else:
            selected_frames = frame_variance

        faces_detected = False
        count = 0
        for _, frame in selected_frames:
            faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5)
            for (x, y, w, h) in faces:
                faces_detected = True
                face = frame[y:y + h, x:x + w]
                resized_face = cv2.resize(face, (224, 224))
                output_path = os.path.join(output_folder, f"{label}_{video_name}_face_{count}.jpg")
                cv2.imwrite(output_path, resized_face)
                count += 1

        if not faces_detected:
            print(f"No faces detected in {video_path}")

    except Exception as e:
        print(f"An error occurred while processing {video_path}: {e}")


In [11]:

# Example usage:
real_video_path = "original"
fake_video_path = "fake"
og_folder = "images/real"
fake_folder = "images/fake"

# Extract faces from real videos
# for video_file in os.listdir(real_video_path):
#     video_path = os.path.join(real_video_path, video_file)
#     # print(f"Processing video: {video_path}")
#     extract_faces_with_variance(video_path, og_folder, label="real")

# Extract faces from fake videos
for video_file in os.listdir(fake_video_path):
    video_path = os.path.join(fake_video_path, video_file)
    extract_faces_with_variance(video_path, fake_folder, label="fake")

## Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Image dimensions
img_width, img_height = 64, 64  # Adjust based on your dataset

train_data_dir = 'images/'  # Path to the parent folder containing 'original' and 'fake' folders
batch_size = 32

# Data generator for training
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # 20% of data for validation

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation')

# CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # use 'softmax' for more than two classes
])


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

# Save the model
model.save('deepfake_detection_model.h5')

## Verification

In [15]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import load_model

def extract_frames(video_path, size=(64, 64)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, size)
        frame = img_to_array(frame)
        frame = np.expand_dims(frame, axis=0)
        frames.append(frame)
    cap.release()
    return np.vstack(frames)

def predict_video(video_path, model):
    frames = extract_frames(video_path)
    # Assuming that your model was trained with normalized images
    frames = frames.astype('float32') / 255.0
    predictions = model.predict(frames)
    avg_prediction = np.mean(predictions)
    return 'FAKE' if avg_prediction > 0.5 else 'REAL'

# Load your trained model
model = load_model('deepfake_detection_model.h5')

# Path to the new video
new_video_path = 'id9_0009.mp4'

# Predict
video_label = predict_video(new_video_path, model)
print(f"The video is predicted as: {video_label}")


10/10 [==============================] - 0s 16ms/step
The video is predicted as: REAL
